In [ ]:
import panel as pn
import param
import os
from io import BytesIO
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, Params
from sklearn.model_selection import train_test_split

# Code libraries
import math
import time
import numpy as np
import numpy.matlib
import pandas as pd
import random
import sys
from copy import deepcopy


pn.extension(sizing_mode='stretch_width') 
hv.extension('bokeh')

In [ ]:
CSV_PATH = os.path.join(os.getcwd(), 'data')

DATA_DIR = os.listdir(CSV_PATH)
ALGORITHMS = ['CRO-ELM', 'PSO-ELM', 'GA-ELM']

#### Configuration Class

In [ ]:
class Configuration(param.Parameterized):
    # General parameters
    database        = param.Selector(objects=DATA_DIR, default=DATA_DIR[0], precedence=1)
    algorithm       = param.Selector(objects= ALGORITHMS, default=ALGORITHMS[0], precedence=1)
    seed            = param.Parameter(default=1234, doc="Random number generator seed", precedence=1)
    generations     = param.Integer(default=10, bounds=(1, 100), precedence=1)
    population_size = param.Number(default=10, bounds=(1, 200), step=1, precedence=1)
    hidden_neurons  = param.Selector(objects=[1] + [x for x in range(50,1001,50)], default=1, precedence=1)
    complexity      = param.Selector(objects=[1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3], default=1, precedence=1)
    
    # CRO parameters
    larvae_attempts       = param.Integer(default=3, bounds=(1, 10), step=1, doc="Number of larvae attempts", precedence=-1)
    occupied_corals       = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Fraction of occupied corals", precedence=-1)
    broadcast_fraction    = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Fraction of broadcast larvae", precedence=-1)
    asexual_fraction      = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Fraction of asexual larvae", precedence=-1)
    asexual_probability   = param.Number(default=0.5, bounds=(0, 1), step=0.001, doc="Probability of asexual larvae", precedence=-1)
    predation_fraction    = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Fraction of larvae lost to predation", precedence=-1)
    predation_probability = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Probability of predation", precedence=-1)
    
    # GA parameters
    mutation_rate = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Mutation rate", precedence=-1)
    crossover_rate = param.Number(default=0.5, bounds=(0, 1), step=0.01, doc="Crossover rate", precedence=-1)

    # PSO parameters
    max_inertia = param.Number(default=0.5, bounds=(0, 1), step=0.01, label="Maximum Inertia weight (w_max)", precedence=-1)
    min_inertia = param.Number(default=0.5, bounds=(0, 1), step=0.01, label="Minimum Inertia weight (w_min)", precedence=-1)
    cognitive = param.Number(default=0.5, bounds=(0, 2), step=0.01, label="Cognitive weight (c1)", precedence=-1)
    social = param.Number(default=0.5, bounds=(0, 2), step=0.01, label="Social weight (c2)", precedence=-1)

    def __init__(self):
        super().__init__()
        self.update_precedences() # Initialize precedence        

    def view(self):
        return pn.Column(self.param)

    @param.depends('algorithm', watch=True)
    def update_precedences(self):
      cro_precedence = 2 if self.algorithm == 'CRO-ELM' else -1
      ga_precedence = 2 if self.algorithm == 'GA-ELM' else -1
      pso_precedence = 2 if self.algorithm == 'PSO-ELM' else -1

      self.param.larvae_attempts.precedence = cro_precedence
      self.param.occupied_corals.precedence = cro_precedence
      self.param.broadcast_fraction.precedence = cro_precedence
      self.param.asexual_fraction.precedence = cro_precedence
      self.param.asexual_probability.precedence = cro_precedence
      self.param.predation_fraction.precedence = cro_precedence
      self.param.predation_probability.precedence = cro_precedence

      self.param.mutation_rate.precedence = ga_precedence
      self.param.crossover_rate.precedence = ga_precedence

      self.param.max_inertia.precedence = pso_precedence
      self.param.min_inertia.precedence = pso_precedence
      self.param.cognitive.precedence = pso_precedence
      self.param.social.precedence = pso_precedence 

    def reset(self):
        self.database = DATA_DIR[0]
        self.algorithm = ALGORITHMS[0]
        self.seed = 1234
        self.generations = 10
        self.population_size = 10
        self.hidden_neurons = 1
        self.complexity = 1
        self.larvae_attempts = 3
        self.occupied_corals = 0.5
        self.broadcast_fraction = 0.5
        self.asexual_fraction = 0.5
        self.asexual_probability = 0.5
        self.predation_fraction = 0.5
        self.predation_probability = 0.5
        self.mutation_rate = 0.5
        self.crossover_rate = 0.5
        self.max_inertia = 0.5
        self.min_inertia = 0.5
        self.cognitive = 0.5
        self.social = 0.5   
        self.update_precedences()  

In [ ]:
# Create the configuration
configuration = Configuration()

#### Bio Classes

In [ ]:
# Individual class for the genetic algorithm.
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (1D list): The chromosome of the individual in [0,1] interval.
    weights (2D list): The weights of the individual in U[-1,1] interval.
    bias (1D list): The bias of the individual in U[-1,1] interval.
    fitness (float): The fitness of the individual.
  """

  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray,  fitness:float = math.inf):
    self._chromosome = chromosome
    self._weights = weights
    self._bias = bias
    
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness

    self._needs_update = True # Flag to indicate if the individual's fitness needs to be updated.
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Biases: {self._bias}
        Fitness: {self._fitness}
        Needs update: {self._needs_update}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights
  
  @property
  def bias(self):
    return self._bias

  @property
  def fitness(self):
    return self._fitness
    
  @property
  def needs_update(self):
    return self._needs_update

  @chromosome.setter
  def chromosome(self, chromosome):
    self._chromosome = chromosome

  @weights.setter
  def weights(self, weights):
    self._weights = weights

  @bias.setter
  def bias(self, bias):
    self._bias = bias

  @fitness.setter
  def fitness(self,f):
    if f < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = f

  @needs_update.setter
  def needs_update(self, needs_update):
    self._needs_update = needs_update

  @staticmethod
  def create_individual(K:int = 0, D:int = 0):
    """
    Generates a random individual.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Individual: A random individual.
    """

    chromosome= np.random.randint(low= 0, high= 2, size = K) # 0 or 1 
    weights= np.random.uniform(low= -1, high= 1, size=(K,D)) # U[-1,1]
    bias= np.random.uniform(low= -1, high= 1, size=(1,D)) # U[-1,1]
    return Individual(chromosome, weights, bias)

# Particle class for the particle swarm optimization.
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray, fitness:float = math.inf):
    super().__init__(chromosome, weights, bias, fitness)
    
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_bias_position = bias # By default, the best bias position is the starting bias position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._bias_velocity = np.zeros(bias.shape) # All swarm's bias velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual bias velocity: {self._bias_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_bias_position(self):
    return self._best_bias_position

  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def bias_velocity(self):
    return self._bias_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f

  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_bias_position.setter
  def best_bias_position(self,b):
    self._best_bias_position = b

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

  @weights_velocity.setter
  def weights_velocity(self,w):
    self._weights_velocity = w

  @bias_velocity.setter
  def bias_velocity(self,b):
    self._bias_velocity = b
    
  @chromosome_velocity.setter
  def chromosome_velocity(self,c):
    self._chromosome_velocity = c

  @staticmethod
  def create_particle(K:int = 0, D:int = 0):
    """
    Generates a random particle.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Particle: A random particle.
    """
    chromosome= np.random.randint(low= 0, high= 2, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Particle(chromosome, weights, bias)

# Larvae class for the coral reef optimization.
class Larvae(Individual):
  """
  A class that represents an individual solution in CRO problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    attempts  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray, fitness:float = math.inf, attempts:int = 0):
    super().__init__(chromosome, weights, bias, fitness)
    self._attempt = attempts
  
  def __str__(self):
    return f"""
    Larvae:
        Fitness: {self.fitness}
        Attempts: {self._attempt}
        Feature-selection: {self.chromosome}
        Weights: {self.weights}
        Bias: {self.bias}
    """
  
  @property
  def attempts(self):
    return self._attempt
  
  @attempts.setter
  def attempts(self,a):
    self._attempt = a
  
  @staticmethod
  def create_larvae(K:int = 0, D:int = 0, attempts:int = 0):
    """
    Generates a random larvae.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      Attempts (int): The number of attempts.

    Returns:
      Larvae: A random larvae.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Larvae(chromosome, weights, bias, attempts = attempts)

# Population class for the GA.
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    genes_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list.
    add_gene_to_list_at_index(Individual): Adds an individual to the gene_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = np.empty(size, dtype=Individual)
    else:
      if is_swarm == True:
        self._genes_list = np.array([Particle.create_particle(K,D) for _ in range(size)]) 
      else:
        self._genes_list = np.array([Individual.create_individual(K,D) for _ in range(size)]) 
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def best_gene(self):
    return self._best_gene

  @genes_list.setter
  def genes_list(self, genes_list):
    self._genes_list = genes_list
    
  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen
  
  @staticmethod
  def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    """
    Generates a random population.

    Args:
      size (int): The size of the population.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the population will be empty.

    Returns:
      Population: A random population.
    """
    return Population(size, K, D, is_empty, is_swarm)

  def add_gene_to_list_at_index(cls,gen, index:int):
    np.put(cls._genes_list, index, gen)

  def insert_best_gene(cls,gen):
    cls.best_gene = gen
    cls._genes_list[0] = gen  # Insert the best gene at the beginning of the list

# Swarm class for the PSO.
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """
  
  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    super().__init__(size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None
    self._global_best_bias = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best bias: {self._global_best_bias}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @property
  def global_best_bias(self):
    return self._global_best_bias

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

  @global_best_bias.setter
  def global_best_bias(self,b):
    self._global_best_bias = b

  @staticmethod
  def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    """
    Generates a random swarm.

    Args:
      size (int): The size of the swarm.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the swarm will be empty.

    Returns:
      Swarm: A random swarm.
    """
    return Swarm(size, K, D, is_empty)

# Reef class for the CRO.
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    free_occupied_rate (float): The rate of occupied individuals in the reef.
    corals_list (numpy.ndarray): The list of corals in the reef.
    best_coral (Individual): The best individual of the population.
    sorted_indexes (numpy.ndarray): The indexes of the sorted corals_list.
    
  Methods:
  -----------
  """

  def __init__(self, size:int, rate:float, K:int, D:int, larvaes_attempts:int):
    self._size = size
    self._free_occupied_rate = rate
    self._best_coral = None
    self._corals_list = np.full(shape = [size], fill_value = None) # Empty reef
    self._sorted_indexes = np.array([i for i in range(size)]) # Array with indexes of corals
    self._larvaes_attempts = larvaes_attempts

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._corals_list[:occupiedHoles] = np.array([Larvae.create_larvae(K,D, larvaes_attempts) for _ in range(occupiedHoles)]) # Fill the reef with individuals

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._corals_list}
        Best coral: {self._best_coral}
        Larvaes attempts: {self._larvaes_attempts}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def corals_list(self):
    return self._corals_list

  @property
  def best_coral(self):
    return self._best_coral

  @property
  def sorted_indexes(self):
    return self._sorted_indexes

  @property
  def larvaes_attempts(self):
    return self._larvaes_attempts
  
  @corals_list.setter
  def corals_list(self,corals_list):
    self._corals_list = corals_list

  @best_coral.setter
  def best_coral(self,coral):
    self._best_coral = coral
  
  @staticmethod
  def create_reef(size:int, rate:float, K:int, D:int, larvaes_attempts:int):
    """
    Generates a reef.

    Args:
      size (int): The size of the reef.
      rate (float): The rate of occupied corals in the reef.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      Larvae_attempts (int): The number of attempts of a newborn larvae.

    Returns:
      Reef: A reef.
    """
    return Reef(size, rate, K, D, larvaes_attempts)

  def insert_new_larvae_in_hole(cls, larvae: Larvae, hole_index:int):
    cls.corals_list[hole_index] = larvae

  def remove_coral_from_hole(cls, hole_index:int):
    cls.corals_list[hole_index] = None
  
  def sort_by_fitness(cls):
    # Sort best_indexes by fitness from corals in corals_list, if corals_list[i] is None, then corals_list[i] is the worst
    cls._sorted_indexes = np.argsort([coral.fitness if coral is not None else math.inf for coral in cls._corals_list])

#### ELM

In [ ]:
''' Train ELM model and output the results '''
def train_model_and_output_results(best_weights:np.ndarray, best_chromosome:np.ndarray, best_bias:np.ndarray, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray) -> float:
  # Weights with feature selection
  w = np.multiply(best_weights, best_chromosome[:, np.newaxis])

  # Amplify the matrix to the size of Xtraining and Xtestwith repmat
  BiasTrainingMatrix = np.matlib.repmat(best_bias, trainingX.shape[0], 1)
  BiasTestMatrix = np.matlib.repmat(best_bias, testX.shape[0], 1)

  # H (Sigmoide function) 
  H_training = 1 / (1 + np.exp(-(np.dot(trainingX, w) + BiasTrainingMatrix)))
  H_test = 1 / (1 + np.exp(-(np.dot(testX, w) + BiasTestMatrix)))

  # Beta with regularization, complete formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
  aux = np.dot(H_training.T, H_training)
  delta = np.identity(D) / C
  Beta = np.dot(np.linalg.pinv(delta +  aux), np.dot(H_training.T, trainingY))

  # Prediction
  predicted_labels = np.dot(H_test, Beta)
  predicted_labels = np.rint(predicted_labels) # @ also computes the dot product
  
  correct_prediction = np.sum(predicted_labels == testY)

  CCR = correct_prediction / testY.shape[0]
  return CCR * 100

'''Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model'''
def compute_individual_fitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  # If the individual's fitness has already been computed, return it.
  if not individual.needs_update:
      return individual.fitness

  # Weights with feature selection 
  individualWeights = np.multiply(individual.weights, individual.chromosome[:, np.newaxis])

  # Amplify the bias matrix to the size of Xtraining and Xtestwith repmat
  BiasTrainingMatrix = np.matlib.repmat(individual.bias, trainingX.shape[0], 1)
  BiasTestMatrix = np.matlib.repmat(individual.bias, testX.shape[0], 1)

  # H (Sigmoide function) 
  H_training = 1 / (1 + np.exp(-(np.dot(trainingX, individualWeights) + BiasTrainingMatrix)))
  H_test = 1 / (1 + np.exp(-(np.dot(testX, individualWeights) + BiasTestMatrix)))

  # Beta with regularization, complete formula: inv( (eye(D) ./ C) + aux) * H_Training' * Ytraining;
  aux = np.dot(H_training.T, H_training)
  delta = np.identity(D) / C
  Beta = np.dot(np.linalg.pinv(delta +  aux), np.dot(H_training.T, trainingY))

  # Output
  Y_predicted = np.dot(H_test, Beta)
  fitness = np.linalg.norm(Y_predicted - testY)
  
  # Update the individual's fitness and return it.
  individual.fitness = fitness
  individual.needs_update = False
 
'''Computes and udpates the fitnesses of a given Population'''
def compute_population_fitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for individual in population.genes_list:
    compute_individual_fitness(individual, D, C, trainingX, trainingY, testX, testY) # Compute fitness

    # Update best gene in Population
    if population.best_gene is None:
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        population.best_gene = individual
  # end for

'''Computes and udpates the fitnesses of a given Swarm'''
def compute_swarm_fitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for index, particle in enumerate(swarm.genes_list):  

    # Evaluate the particle's fitness    
    compute_individual_fitness(particle, D, C, trainingX, trainingY, testX, testY) # Compute fitness

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      # Update personal bests
      particle.best_fitness = particle.fitness      
      particle.best_weights_position = particle.weights
      particle.best_bias_position = particle.bias
      particle.best_chromosome_position = particle.chromosome
      
      # Update global best
      if swarm.best_gene is None:
        swarm.best_gene = particle
      else:
        if particle.fitness < swarm.best_gene.best_fitness:
          swarm.best_gene = particle
  # end for

  # Update Swarm's best
  if swarm.best_gene.best_fitness < swarm.global_best_fitness:
    swarm.global_best_fitness = swarm.best_gene.best_fitness
    swarm.global_best_weights = swarm.best_gene.best_weights_position
    swarm.global_best_bias = swarm.best_gene.best_bias_position
    swarm.global_best_chromosome = swarm.best_gene.best_chromosome_position

'''Computes and udpates the fitnesses of a given Reef'''
def compute_reef_fitness(reef: Reef, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for coral in reef.corals_list:  
    if coral is None: # If the coral is empty, skip it
      continue

    compute_individual_fitness(coral, D, C, trainingX, trainingY, testX, testY) 
    
    # Update best gene in Reef
    if reef.best_coral is None:
      reef.best_coral = coral
    else:
      if coral.fitness < reef.best_coral.fitness:
        reef.best_coral = coral

#### GA

In [ ]:
''' Takes a newly created individual and mutates it.
    1. The individual is mutated by mutating the chromosome, weights and bias.
    2. The individual fitness is not calculated yet and the needs_update flag is already set to true.'''
def internal_reproduction(gen: Individual):
  # Mutate chromosome
  mutate_chromosome_point = np.random.randint(0, gen.chromosome.shape[0]) # Choose a random feature to mutate
  gen.chromosome[mutate_chromosome_point] = 1 - gen.chromosome[mutate_chromosome_point] # Activate or deactivate the feature
  
  # Mutate weights
  total_rows = gen.weights.shape[0] 
  total_columns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

  # Mutate bias
  mutate_bias_point = np.random.randint(0, gen.bias.shape[1]) # Choose a random column to mutate
  gen.bias[0,mutate_bias_point] = np.random.uniform(-1,1) # Mutate the column

''' Takes two individuals and creates two new individuals by combining their chromosomes, weights and bias.'''
def external_reproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, father.chromosome.shape[0])
  crossover_column = np.random.randint(0, father.weights.shape[1])
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1),
                           bias = np.concatenate((father.bias[:,:crossover_column], mother.bias[:,crossover_column:]), axis=1) )
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1),
                           bias = np.concatenate((mother.bias[:,:crossover_column], father.bias[:,crossover_column:]), axis=1) )
                       
  return offspring1, offspring2

''' Takes a population and returns a new population with the fixed size by roulette wheel selection.'''
def roulette_wheel_selection(population:Population):
    if population.size == 0:
        raise ValueError("Population is empty.")
    # Create an empty population for the chosen individuals of the Roulette Wheel
    roulettePopulation = Population.create_population(population.size, is_empty=True)
    roulettePopulation.insert_best_gene(population.best_gene) # Make sure we don't lose our best gene in the roulette 

    # Total population fitness (S)
    S = np.sum([individual.fitness for individual in population.genes_list])

    # Population chromosomes' relative probabilities
    rel_prob = [individual.fitness/S for individual in population.genes_list]

    # Create the list of accumulated relative probabilities
    acc_rel_prob = np.cumsum(rel_prob)

    for idx in range(1, roulettePopulation.size): # -1 because we already inserted the best gene
        r = np.random.uniform() 
        # Find the first index for which q_i < r
        for index, acc_prob in enumerate(acc_rel_prob): 
            if r < acc_prob:
                roulettePopulation.add_gene_to_list_at_index(population.genes_list[index], idx)
                break
    # end roulette population loop
    return roulettePopulation
  
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''
def reproduce_population(population: Population, CROSSOVER_PROBABILITY:float, MUTATION_PROBABILITY:float):
  childs = np.empty(shape=0, dtype=Individual) # list to store future offsprings
  gene_list_size = len(population.genes_list)

  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() < CROSSOVER_PROBABILITY):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        random_mother_index = np.random.randint(0,gene_list_size)
      #end while
      
      # Crossover parents
      mother = population.genes_list[random_mother_index]
      offspring1, offspring2 = external_reproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      if(np.random.random() < MUTATION_PROBABILITY): 
        internal_reproduction(offspring1) # Mutate offspring 1

      if(np.random.random() < MUTATION_PROBABILITY):
        internal_reproduction(offspring2) # Mutate offspring 2

      # Add offsprings to the childs list
      childs = np.append(childs, [offspring1, offspring2])
  
  # Extend the population with the new childs
  population.genes_list = np.concatenate( (population.genes_list, childs) )
  return population

  ''' This functions apply the Evolutionary-based Genetic Algorithm to a given population '''

#### PSO

In [ ]:
def update_velocities_and_positions_pso(swarm: Swarm, w:float, c1:float, c2:float):
  # Random numbers
  r1 = np.random.uniform()
  r2 = np.random.uniform()

  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm
    # Weights update 
    particle.weights_velocity = ( 
                                  w * particle.weights_velocity +
                                  c1*r1*(particle.best_weights_position - particle.weights) +
                                  c2*r2*(swarm.best_gene.best_weights_position - particle.weights)
                                )

    particle.weights += particle.weights_velocity # Update to next position
    
    # Bias update
    particle.bias_velocity = ( 
                                w * particle.bias_velocity +
                                c1*r1*(particle.best_bias_position - particle.bias) +
                                c2*r2*(swarm.best_gene.best_bias_position - particle.bias)
                             )

    particle.bias += particle.bias_velocity # Update to next position
    
    # Chromosome update
    next_velocity= (
                      w * particle.chromosome_velocity +
                      c1*r1*(particle.best_chromosome_position - particle.chromosome) +
                      c2*r2*(swarm.best_gene.best_chromosome_position - particle.chromosome)
                   ) 

    # Prevent velocities from going out of bounds
    np.clip(next_velocity, -6, 6, out=particle.chromosome_velocity)
    velocityProbability = 2 / np.pi * np.arctan((np.pi*0.5)*particle.chromosome_velocity) # PSO-ELM paper (Mirjalili and Lewis, 2013)
    particle.chromosome = np.where(np.random.random() < velocityProbability, 1, 0)

    # Particle has been updated
    particle.needs_update = True
  # end particle loop
  return swarm

#### CRO

In [ ]:
def broadcast(father:Larvae, mother:Larvae, attempts:int):
    crossover_point_chromosome = np.random.randint(0, father.chromosome.shape[0])
    crossover_column = np.random.randint(0, father.weights.shape[1])

    # Offsprings are created by combining father and mother chromosomes and weights
    new_larvae = Larvae( chromosome = np.concatenate((father.chromosome[:crossover_point_chromosome], mother.chromosome[crossover_point_chromosome:])), 
                         weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1),
                         bias = np.concatenate((father.bias[:,:crossover_column], mother.bias[:,crossover_column:]), axis=1),
                         attempts= attempts)
                         
    # Larvae created by broadcast has needs_update flag set to True by default
    return new_larvae

def brooding(larvae: Larvae, attempts:int):

    # Auxiliar larvae to mutate instead of the original
    aux_larvae = deepcopy(larvae)

    # Mutate chromosome
    mutate_chromosome_point = np.random.randint(0, aux_larvae.chromosome.shape[0]) # Choose a random feature to mutate
    aux_larvae.chromosome[mutate_chromosome_point] = 1 - aux_larvae.chromosome[mutate_chromosome_point] # Activate or deactivate the feature

    # Mutate weights
    total_rows = aux_larvae.weights.shape[0] 
    total_columns = aux_larvae.weights.shape[1] 
    mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate

    larvae.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

    # Mutate bias
    mutate_bias_point = np.random.randint(0, aux_larvae.bias.shape[1]) # Choose a random column to mutate
    aux_larvae.bias[0,mutate_bias_point] = np.random.uniform(-1,1) # Mutate the column

    # Because the larvae was created by brooding, it needs to be updated and reset attempts
    aux_larvae.needs_update = True
    aux_larvae.attempts = attempts    
    return aux_larvae

def split_reef_candidates(candidates:list, fraction:float):
    number_of_candidates = len(candidates)
    split_point = int(number_of_candidates * fraction)

    # Split candidates in two lists
    broadcast_candidates = candidates[:split_point]
    brooding_candidates = candidates[split_point:]
    
    # If broadcast candidates are odd, add one more candidate to the list and remove it from the brooding candidates
    if len(broadcast_candidates) % 2 != 0:
        broadcast_candidates.append(candidates[split_point])
        brooding_candidates.remove(candidates[split_point])

    return [broadcast_candidates, brooding_candidates]

def larvae_setting(pool:np.ndarray, reef:Reef, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
    alive_larvae = []
    # Try to settle the larvaes from the pool
    for i in range(len(pool)):
        # Compute fitness of the larvae
        compute_individual_fitness(pool[i], OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

        # Try to settle the larvae in a random hole
        random_hole = np.random.randint(0, reef.size)
        if reef.corals_list[random_hole] is None or pool[i].fitness < reef.corals_list[random_hole].fitness:
            reef.insert_new_larvae_in_hole(pool[i], random_hole)
            alive_larvae.append(pool[i])
        else:
            if pool[i].attempts > 0:
                pool[i].attempts -= 1
                alive_larvae.append(pool[i])
    # end for         
    pool = alive_larvae # Update pool

def predation(reef:Reef, predation_fraction:int, predation_probability:float):
  # Predate from the last corals with worse health
  for index in range(reef.size - predation_fraction, reef.size):
    if np.random.uniform(0, 1) < predation_probability:      
      reef.corals_list[reef.sorted_indexes[index]] = None

def reproduce_reef(sexual_corals:list, f_broadcast:float, new_larvaes_attempts:int):
  sexual_larvaes_pool = np.array([]) # pool of candidates from broadcast and brooding

  # Split reproduction sets
  broadcast_set, brooding_set = split_reef_candidates(sexual_corals, f_broadcast)

  # Broadcast    
  if len(broadcast_set) > 0: # If broadcast set is not empty, crossover
    while len(broadcast_set) > 0:
      # Select random father and a mother to crossover
      [father_candidate, mother_candidate] = np.random.choice(broadcast_set, 2, replace=False)
      broadcast_larvae = broadcast(father = father_candidate, 
                                   mother =  mother_candidate,
                                   attempts= new_larvaes_attempts)

      # Add offspring to the pool
      sexual_larvaes_pool = np.append(sexual_larvaes_pool,broadcast_larvae) # Two parents reproduce only one coral larva
      
      # Remove fathers of the reproduction list
      broadcast_set.remove(father_candidate)
      broadcast_set.remove(mother_candidate)
  ##

  #  Brooding
  if len(brooding_set) > 0: # If broadcast set is not empty, mutate every individual
    for c in brooding_set:
      brooding_larvae = brooding(c, new_larvaes_attempts)
      sexual_larvaes_pool = np.append(sexual_larvaes_pool, brooding_larvae) # Add the mutated coral to the pool
  ##
  return sexual_larvaes_pool

#### DATA PROCESSING

In [ ]:
def process_data(df:pd.DataFrame):
    end = df.shape[1]

    # - - - DATAFRAME CONSTANTS - - - #
    X = df.iloc[:, 0:end-1].values
    Y = df.iloc[:, end-1].values

    J = len(np.unique(Y)) # Number of classes
    N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features

    # - - - DATAFRAME PARTITIONS - - - #
    X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Scaling X (min-max normalization)
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # First partition with 20% test data
    X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Second partition with 20% validation data
    # - - -    - - - #
    return X_trainVal, X_testVal, Y_trainVal, Y_testVal, X_train, Y_train, X_test, Y_test, J, N, K

#### ACTION BUTTONS

In [ ]:
from holoviews import opts
from holoviews.streams import Pipe, Buffer

class ActionButtons(param.Parameterized):
    # Buttons
    reset_action = param.Action(lambda self: self.reset(), label='Reset Parameters', precedence=1)
    run_action = param.Action(lambda self: self.run(), label='\u25b6 Run Algorithm', precedence=1)
    next_generation_action = param.Action(lambda self: self.next_generation(), label='Next Generation', precedence=1)
    #save_action = param.Action(lambda self: self.save(), label='Export Results', precedence=1)


    def __init__(self, configuration:Configuration):
        super().__init__()
        
        self.configuration = configuration
        np.random.seed(self.configuration.seed)
        random.seed(self.configuration.seed)

        # Algorithm parameters
        self.actual_generation = 0
        self.running_algorithm_population = None
        
        # Plot parameters
        self.fitness_dataframe = pd.DataFrame({'Generations': [], 'Fitness': []}, columns=['Generation', 'Fitness'])
        self.buffer = Buffer(data=self.fitness_dataframe, length=100, index=False)
        self.curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.buffer])
        self.points_dmap = hv.DynamicMap(hv.Points, streams=[self.buffer])
        self.plot = (self.curve_dmap * self.points_dmap).opts(
            opts.Curve(color='blue', line_width=2),
            opts.Points(color='orange', line_color='black', size=5))

        # Export parameters, by default, the export dataframe is empty with no parameters related to the algorithm
        self.results_dataframe = pd.DataFrame(columns=['Dataset', 'Generations', 'Population Size', 'Selected Algorithm', 'D', 'C'])


    def view(self):
        return pn.Column(self.plot.opts(width=800,height=300), self.param.run_action, self.param.next_generation_action,
                         pn.Row(self.param.reset_action))
    
    ''' This functions apply the Evolutionary-based Genetic Algorithm to a given population with the given parameters'''
    def ga(self, population: Population, crossover_prob:float, mutation_prob:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
        # Reproduction
        population = reproduce_population(population, crossover_prob, mutation_prob)

        # Compute Fitness
        compute_population_fitness(population= population, D= OPTIMAL_D, 
                                C= OPTIMAL_C, trainingX= trainX,
                                trainingY= trainY, testX= testX,
                                testY= testY)

        # Roulette Selection
        population = roulette_wheel_selection(population)
        return population

    ''' This method apply the Swarm-based algorithm of PSO to a given swarm'''
    def pso(self, swarm: Swarm, actual_t:int, max_generations:int, w_max:float,  w_min:float, c1:float, c2:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
        # Linearly decrease the inertia weight
        w = w_max - (w_max - w_min) * actual_t / max_generations
        # Evaluate Particles' Fitness and update Pb_i and Gb
        compute_swarm_fitness(swarm, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

        # Update Velocity and Position of each particle
        update_velocities_and_positions_pso(swarm, w, c1, c2)
        return swarm

    ''' This function applies the Evolutionary-based Coral Reef Optimization algorithm to a given reef with the given parameters'''
    def cro(self, reef:Reef, f_broadcast:float, f_asexual:float, f_predation:float, asexual_probability:float, predation_probability:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
        pool = np.array([]) # Empty pool

        # Constants
        predation_fraction = int(reef.size * f_predation) # Number of corals to depredate at each generation
        budding_corals = int(len(reef.corals_list) * f_asexual) # Number of corals to reproduce by budding at each generation
        
        # Evaluate reef at the beginning
        compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

        # Non empty corals in the reef
        non_empty_corals = [c for c in reef.corals_list if c is not None]
        
        # Sort the reef by corals' health function
        reef.sort_by_fitness()

        # Asexual reproduction if the probability is met and there are corals to reproduce
        if np.random.uniform(0, 1) < asexual_probability and len(non_empty_corals) > 0 and budding_corals > 0: 
            for index in reef.sorted_indexes[:budding_corals]: 
                if reef.corals_list[index] is not None:
                    asexual_larvae = deepcopy(reef.corals_list[index]) # Budding
                    asexual_larvae.attempts = reef.larvaes_attempts # Reset attempts
                    pool = np.append(pool, asexual_larvae) # Add larvae to the pool
            ##
        
        # Broadcast and brooding
        sexual_larvaes = reproduce_reef(non_empty_corals, f_broadcast, new_larvaes_attempts=reef.larvaes_attempts)
        pool = np.append(pool, sexual_larvaes)
        
        # Larvae settings    
        larvae_setting(pool, reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

        # Predation
        reef.sort_by_fitness() # Sort reef before predation so the worst corals are predated   
        predation(reef, predation_fraction, predation_probability)

        #stream.event(t)

        reef.sort_by_fitness() # Final sort
        reef.best_coral = reef.corals_list[reef.sorted_indexes[0]] # Best coral
        return reef

    def run_algorithms(self, generations:int, next_gen:bool=False):
        # Clear the plot
        if not next_gen:
            self.clear_plot()
            self.actual_generation = 0

        # Run the algorithm
        print(f"Run {self.configuration.algorithm} with {self.configuration.database} database and generations = {self.configuration.generations} and seed = {self.configuration.seed}")

        # Read data from its location
        df = pd.read_csv(os.path.join(CSV_PATH, self.configuration.database), sep=" ", header=None)
        X_trainVal, X_testVal, Y_trainVal, Y_testVal, X_train, Y_train, X_test, Y_test, J, N, K = process_data(df)
        
        if self.configuration.algorithm == 'CRO-ELM':
            # Create reef
            if self.actual_generation == 0:
                self.running_algorithm_population = Reef.create_reef(size=self.configuration.population_size,
                                                                        rate= self.configuration.occupied_corals,
                                                                        K= K,
                                                                        D= self.configuration.hidden_neurons,
                                                                        larvaes_attempts= self.configuration.larvae_attempts)
            # Run the algorithm
            start_cro_time = time.time()

            for t in range(generations):
                # Set seed for reproducibility
                np.random.seed(self.configuration.seed)
                random.seed(self.configuration.seed)
                self.running_algorithm_population = self.cro(reef= self.running_algorithm_population,
                                                            f_broadcast= self.configuration.broadcast_fraction,
                                                            f_asexual= self.configuration.asexual_fraction,
                                                            f_predation= self.configuration.predation_fraction,
                                                            asexual_probability= self.configuration.asexual_probability,
                                                            predation_probability= self.configuration.predation_probability,
                                                            OPTIMAL_D= self.configuration.hidden_neurons,
                                                            OPTIMAL_C= self.configuration.complexity,
                                                            trainX= X_trainVal,
                                                            trainY= Y_trainVal,
                                                            testX= X_testVal,
                                                            testY= Y_testVal)
                generation_data = pd.DataFrame({'Generation': [self.actual_generation], 'Fitness': [self.running_algorithm_population.best_coral.fitness]})
                self.fitness_dataframe = pd.concat([self.fitness_dataframe, generation_data], ignore_index=True)
                self.buffer.send(self.fitness_dataframe)
                self.actual_generation += 1
            # end for

            CRO_CCR = train_model_and_output_results(best_weights= self.running_algorithm_population.best_coral.weights, 
                                                     best_chromosome = self.running_algorithm_population.best_coral.chromosome, 
                                                     best_bias = self.running_algorithm_population.best_coral.bias,
                                                     D = self.configuration.hidden_neurons,
                                                     C = self.configuration.complexity, 
                                                     trainingX = X_train,
                                                     trainingY = Y_train,
                                                     testX = X_test,
                                                     testY = Y_test)
            cro_execution_time = time.time() - start_cro_time
            self.results_dataframe = pd.DataFrame({'Database': [self.configuration.database],
                                                    'Generations': [self.configuration.generations],
                                                    'Population size': [self.configuration.population_size],
                                                    'Algorithm': [self.configuration.algorithm],
                                                    'CCR': [CRO_CCR],
                                                    'D': [self.configuration.hidden_neurons],
                                                    'C': [self.configuration.complexity],
                                                    'Occupied corals': [self.configuration.occupied_corals],
                                                    'Larvae attempts': [self.configuration.larvae_attempts],
                                                    'Broadcast fraction': [self.configuration.broadcast_fraction],
                                                    'Asexual fraction': [self.configuration.asexual_fraction],
                                                    'Predation fraction': [self.configuration.predation_fraction],
                                                    'Asexual probability': [self.configuration.asexual_probability],
                                                    'Predation probability': [self.configuration.predation_probability],
                                                    'Execution time': [cro_execution_time],})

            print(f"CRO execution time: {cro_execution_time} seconds")
            return cro_execution_time
            
        elif self.configuration.algorithm == 'GA-ELM':
            # Create population
            if self.actual_generation == 0:
                self.running_algorithm_population = Population.create_population(size= self.configuration.population_size,
                                                                                    K= K,
                                                                                    D= self.configuration.hidden_neurons)
            
            # Run the algorithm
            start_ga_time = time.time()

            for t in range(generations):
                np.random.seed(self.configuration.seed)
                random.seed(self.configuration.seed)
                self.running_algorithm_population = self.ga(population= self.running_algorithm_population,
                                                            crossover_prob= self.configuration.crossover_rate,
                                                            mutation_prob= self.configuration.mutation_rate,
                                                            OPTIMAL_D= self.configuration.hidden_neurons,
                                                            OPTIMAL_C= self.configuration.complexity,
                                                            trainX= X_train,
                                                            trainY= Y_train,
                                                            testX= X_test,
                                                            testY= Y_test)

                generation_data = pd.DataFrame({'Generation': [self.actual_generation], 'Fitness': [self.running_algorithm_population.best_gene.fitness]})
                self.fitness_dataframe = pd.concat([self.fitness_dataframe, generation_data], ignore_index=True)
                self.buffer.send(self.fitness_dataframe)
                self.actual_generation += 1
            # end for
            GA_CCR = train_model_and_output_results(best_weights= self.running_algorithm_population.best_gene.weights, 
                                                    best_chromosome = self.running_algorithm_population.best_gene.chromosome, 
                                                    best_bias= self.running_algorithm_population.best_gene.bias,
                                                    D = self.configuration.hidden_neurons,
                                                    C = self.configuration.complexity, 
                                                    trainingX = X_train,
                                                    trainingY = Y_train,
                                                    testX = X_test,
                                                    testY = Y_test)
            ga_execution_time = time.time() - start_ga_time
            self.results_dataframe = pd.DataFrame({'Database': [self.configuration.database],
                                                    'Generations': [self.configuration.generations],
                                                    'Population size': [self.configuration.population_size],
                                                    'Algorithm': [self.configuration.algorithm],
                                                    'CCR': [GA_CCR],
                                                    'D': [self.configuration.hidden_neurons],
                                                    'C': [self.configuration.complexity],
                                                    'Crossover rate': [self.configuration.crossover_rate],
                                                    'Mutation rate': [self.configuration.mutation_rate],
                                                    'Execution time': [ga_execution_time],})

            print(f"GA execution time: {ga_execution_time} seconds")  
            return ga_execution_time

        elif self.configuration.algorithm == 'PSO-ELM':
            # Create swarm
            if self.actual_generation == 0:
                self.running_algorithm_population = Swarm.create_swarm(size= self.configuration.population_size,
                                                                        K= K,
                                                                        D= self.configuration.hidden_neurons)
            
            # Run the algorithm
            start_pso_time = time.time()
            for t in range(generations):
                np.random.seed(self.configuration.seed)
                random.seed(self.configuration.seed)
                self.running_algorithm_population = self.pso(swarm= self.running_algorithm_population,
                                                            actual_t= t,
                                                            max_generations= self.configuration.generations,
                                                            w_max= self.configuration.max_inertia,
                                                            w_min= self.configuration.min_inertia,
                                                            c1= self.configuration.cognitive,
                                                            c2= self.configuration.social,
                                                            OPTIMAL_D= self.configuration.hidden_neurons,
                                                            OPTIMAL_C= self.configuration.complexity,
                                                            trainX= X_train,
                                                            trainY= Y_train,
                                                            testX= X_test,
                                                            testY= Y_test)

                generation_data = pd.DataFrame({'Generation': [self.actual_generation], 'Fitness': [self.running_algorithm_population.global_best_fitness]})
                self.fitness_dataframe = pd.concat([self.fitness_dataframe, generation_data], ignore_index=True)
                self.buffer.send(self.fitness_dataframe)
                self.actual_generation += 1
            # end for

            PSO_CCR = train_model_and_output_results(best_weights= self.running_algorithm_population.global_best_weights,
                                                     best_chromosome = self.running_algorithm_population.global_best_chromosome,
                                                     best_bias= self.running_algorithm_population.global_best_bias,
                                                     D = self.configuration.hidden_neurons,
                                                     C = self.configuration.complexity,
                                                     trainingX = X_train,
                                                     trainingY = Y_train,
                                                     testX = X_test,
                                                     testY = Y_test)
            pso_execution_time = time.time() - start_pso_time
            self.results_dataframe = pd.DataFrame({'Database': [self.configuration.database],
                                                    'Generations': [self.configuration.generations],
                                                    'Population size': [self.configuration.population_size],
                                                    'Algorithm': [self.configuration.algorithm],
                                                    'CCR': [PSO_CCR],
                                                    'D': [self.configuration.hidden_neurons],
                                                    'C': [self.configuration.complexity],
                                                    'W_max': [self.configuration.max_inertia],
                                                    'W_min': [self.configuration.min_inertia],
                                                    'C1': [self.configuration.cognitive],
                                                    'C2': [self.configuration.social],
                                                    'Execution time': [pso_execution_time],})
            print(f"PSO execution time: {pso_execution_time} seconds")
            return pso_execution_time
    
    # Events  
    @param.depends('run_action', watch=True)
    def run(self):
        # Run the algorithm through all the generations
        self.run_algorithms(self.configuration.generations)

    #@param.depends('save_action', watch=True)
    def save(self):
        # Save the configuration
        buffer = BytesIO()
        self.results_dataframe.to_excel(buffer, index=False)
        buffer.seek(0)
        return buffer

    def clear_plot(self):
        # Clear the plot
        self.fitness_dataframe = pd.DataFrame({'Generations': [], 'Fitness': []}, columns=['Generation', 'Fitness'])
        self.buffer.clear()
        #self.curve_dmap.update()
        #self.points_dmap.update()

    @param.depends('reset_action', watch=True)
    def reset(self):
        # Reset the configuration
        self.configuration.reset()
        self.actual_generation = 0
        self.clear_plot()
 

    @param.depends('next_generation_action', watch=True)
    def next_generation(self):
        # Next generation
        self.run_algorithms(1, True)


In [ ]:
# Set seed for reproducibility
np.random.seed(configuration.seed)
random.seed(configuration.seed)
header = pn.pane.Markdown('''
# Bio-Inspired Optimization Algorithms in Extreme Learning Machine
## Instructions: 
1. Click '\u25b6 Begin Improving' button to begin improving for the time on the Time Evolving slider. 
2. Experiment with the sliders to see how the algorithm performs.'''
)

# Create the action buttons
action_buttons = ActionButtons(configuration= configuration)

export_button = pn.widgets.FileDownload(
    callback= action_buttons.save,
    filename='results.xlsx',
    label='Export results')

# Create the app and deploy it
app = pn.Row(pn.Column(header, action_buttons.view(), export_button), configuration.view())


app.show()